# HW 4 - SQL


## Due: Friday, Oct 12, 2018, 11:59:00pm

### Submission instructions
After completing this homework, you will turn in two files via Canvas ->  Assignments -> HW 4:
- Your Notebook, named si330-hw4-YOUR_UNIQUE_NAME.ipynb and
- the HTML file, named si330-hw4-YOUR_UNIQUE_NAME.html

### Name: John Van Dalsem
### Uniqname: jjvandal
### People you worked with: Will Godley, Trevor Woods


## Part 0: Setup and Database

In [3]:
import sqlite3 as sqlite

We have provided you with an SQLite database in `chinook.db`. This database contains the following tables:

- `artists`: Musical artists, each with an artist id and name.
- `albums`: Musical albums. Each album belongs to one artist. However, one artist may have multiple albums.
- `tracks`: Songs. Each track belongs to one album.
- `invoices` and `invoice_items`: Purchase invoices. Each invoice represents a single purchase, associated with one or more invoice items. Each invoice item represents the purchase of a single track, the dollar amount paid, and the quantity.
- `employees`: Employees, each with an employee id, last name, first name, etc. It also has a field named ReportsTo to specify who reports to whom.
- `customers`: Customers.
- `media_types`: Media types, such as MPEG audio file, ACC audio file, etc.
- `genres`: Music types such as rock, jazz, metal, etc.
- `playlists` and `playlist_track`: Musical playlists. Each playlist contains a list of tracks. Each track may belong to multiple playlists. The relationship between the playlists table and tracks table is many-to-many. The playlist_track table is used to reflect this relationship.
 

A schematic view of this database is below:
    
![The chinook database](chinook.png)

### Part 0.1: SQLite DB Browser

As in the lab, we recommend that you use an SQLite GUI, like the [SQLite DB Browser](http://sqlitebrowser.org/), to browse the database and test your queries.

## Part 1: Connect to database

Insert the code to:
- connect to the database in `chinook.db` using a variable called `con`
- set up a row factory so that results from the database are returned as dictionary-like objects.
- and open a cursor to execute queries from (in a variable called `cur`)


In [4]:
con = sqlite.connect('chinook.db')
con.row_factory = sqlite.Row
cur = con.cursor()

## Part 2: Queries!

Let's answer some questions about the database. Below each entry, insert the code to execute a query against the database and print out the answer.

**All questions in this part and the remaining sections of this assignment must be answered using SQL (apart from code to print out the result). You may not further process the data in python except to print the rows returned by your query.**

**2.1** How many artists are in the database?

In [15]:
cur.execute('SELECT COUNT(*) from artists')
for a in cur:
    print(a[0])

275


**2.2** Who are the top ten artists by number of **albums**? Print their names and the number of albums they have. (HINT: use the `LIMIT` clause to get just the top 10; **do not** use list slicing in python).

In [35]:
cur.execute('SELECT artists.Name, COUNT(albums.ArtistId) AS Album_Count FROM artists JOIN albums ON albums.ArtistId = artists.ArtistId GROUP BY artists.Name ORDER BY Album_Count DESC LIMIT 10')
for a in cur:
    print(dict(a))



{'Name': 'Iron Maiden', 'Album_Count': 21}
{'Name': 'Led Zeppelin', 'Album_Count': 14}
{'Name': 'Deep Purple', 'Album_Count': 11}
{'Name': 'Metallica', 'Album_Count': 10}
{'Name': 'U2', 'Album_Count': 10}
{'Name': 'Ozzy Osbourne', 'Album_Count': 6}
{'Name': 'Pearl Jam', 'Album_Count': 5}
{'Name': 'Faith No More', 'Album_Count': 4}
{'Name': 'Foo Fighters', 'Album_Count': 4}
{'Name': 'Lost', 'Album_Count': 4}


**2.3** Who are the top ten artists by number of **tracks**? Print their names and the number of tracks they have.

In [36]:
cur.execute('SELECT artists.Name, COUNT(tracks.TrackId) AS Track_Count FROM artists JOIN albums ON albums.ArtistId = artists.ArtistId JOIN tracks ON tracks.AlbumId = albums.AlbumId GROUP BY artists.Name ORDER BY Track_Count DESC LIMIT 10')
for a in cur:
    print(dict(a))

{'Name': 'Iron Maiden', 'Track_Count': 213}
{'Name': 'U2', 'Track_Count': 135}
{'Name': 'Led Zeppelin', 'Track_Count': 114}
{'Name': 'Metallica', 'Track_Count': 112}
{'Name': 'Deep Purple', 'Track_Count': 92}
{'Name': 'Lost', 'Track_Count': 92}
{'Name': 'Pearl Jam', 'Track_Count': 67}
{'Name': 'Lenny Kravitz', 'Track_Count': 57}
{'Name': 'Various Artists', 'Track_Count': 56}
{'Name': 'The Office', 'Track_Count': 53}


**2.4** How many tracks are in the database?

In [37]:
cur.execute('SELECT COUNT(*) from tracks')
for a in cur:
    print(a[0])

3503


**2.5** What are the top ten albums by number of purchases of any track off the album? For example, if two tracks were purchased off an album, that counts as two purchases for that album. Print the artist name, album, and the number of purchases.

In [69]:
cur.execute('SELECT artists.Name, albums.Title, COUNT(invoice_items.TrackId) AS Number_of_Purchases FROM artists JOIN albums ON albums.ArtistId = artists.ArtistId JOIN tracks ON tracks.AlbumId = albums.AlbumId JOIN invoice_items ON invoice_items.TrackId = tracks.TrackId GROUP BY albums.Title ORDER BY Number_of_Purchases DESC LIMIT 10')
for a in cur:
    print(dict(a))

{'Name': 'Chico Buarque', 'Title': 'Minha Historia', 'Number_of_Purchases': 27}
{'Name': 'Lenny Kravitz', 'Title': 'Greatest Hits', 'Number_of_Purchases': 26}
{'Name': 'Eric Clapton', 'Title': 'Unplugged', 'Number_of_Purchases': 25}
{'Name': 'Titãs', 'Title': 'Acústico', 'Number_of_Purchases': 22}
{'Name': 'Kiss', 'Title': 'Greatest Kiss', 'Number_of_Purchases': 20}
{'Name': 'Creedence Clearwater Revival', 'Title': 'Chronicle, Vol. 2', 'Number_of_Purchases': 19}
{'Name': 'The Who', 'Title': 'My Generation - The Very Best Of The Who', 'Number_of_Purchases': 19}
{'Name': 'Caetano Veloso', 'Title': 'Prenda Minha', 'Number_of_Purchases': 19}
{'Name': 'Os Paralamas Do Sucesso', 'Title': 'Acústico MTV', 'Number_of_Purchases': 18}
{'Name': 'Battlestar Galactica (Classic)', 'Title': 'Battlestar Galactica (Classic), Season 1', 'Number_of_Purchases': 18}


**2.6** Who are the top ten artists by number of purchases of their tracks? Print the artist name and the number of purchases.

In [70]:
cur.execute('SELECT artists.Name, COUNT(invoice_items.TrackId) AS Number_of_Purchases FROM artists JOIN albums ON albums.ArtistId = artists.ArtistId JOIN tracks ON tracks.AlbumId = albums.AlbumId JOIN invoice_items ON invoice_items.TrackId = tracks.TrackId GROUP BY artists.Name ORDER BY Number_of_Purchases DESC LIMIT 10')
for a in cur:
    print(dict(a))

{'Name': 'Iron Maiden', 'Number_of_Purchases': 140}
{'Name': 'U2', 'Number_of_Purchases': 107}
{'Name': 'Metallica', 'Number_of_Purchases': 91}
{'Name': 'Led Zeppelin', 'Number_of_Purchases': 87}
{'Name': 'Os Paralamas Do Sucesso', 'Number_of_Purchases': 45}
{'Name': 'Deep Purple', 'Number_of_Purchases': 44}
{'Name': 'Faith No More', 'Number_of_Purchases': 42}
{'Name': 'Lost', 'Number_of_Purchases': 41}
{'Name': 'Eric Clapton', 'Number_of_Purchases': 40}
{'Name': 'R.E.M.', 'Number_of_Purchases': 39}


**2.7** How many tracks were purchased by customers from each country in the database? Print each country name and the number of tracks.

In [80]:
cur.execute('SELECT invoices.BillingCountry, COUNT(invoice_items.TrackId) AS Purchased_Track_Count FROM invoices JOIN invoice_items ON invoice_items.InvoiceId = invoices.InvoiceId GROUP BY invoices.BillingCountry ORDER BY Purchased_Track_Count DESC LIMIT 10')
for a in cur:
    print(dict(a))

{'BillingCountry': 'USA', 'Purchased_Track_Count': 494}
{'BillingCountry': 'Canada', 'Purchased_Track_Count': 304}
{'BillingCountry': 'Brazil', 'Purchased_Track_Count': 190}
{'BillingCountry': 'France', 'Purchased_Track_Count': 190}
{'BillingCountry': 'Germany', 'Purchased_Track_Count': 152}
{'BillingCountry': 'United Kingdom', 'Purchased_Track_Count': 114}
{'BillingCountry': 'Czech Republic', 'Purchased_Track_Count': 76}
{'BillingCountry': 'Portugal', 'Purchased_Track_Count': 76}
{'BillingCountry': 'India', 'Purchased_Track_Count': 74}
{'BillingCountry': 'Argentina', 'Purchased_Track_Count': 38}


**2.8** What is the *most* popular genre by **number of tracks**? Your SQL query should return **only one row**, giving the genre name and the number of songs in the database in that genre.

In [78]:
cur.execute('SELECT genres.Name, COUNT(tracks.TrackId) AS Number_of_Tracks FROM genres JOIN tracks ON tracks.GenreId=genres.GenreId GROUP BY genres.Name ORDER BY Number_of_Tracks DESC LIMIT 1')
for a in cur:
    print(dict(a))

{'Name': 'Rock', 'Number_of_Tracks': 1297}


**2.9** What is the *least* popular genre by **number of tracks**? Your SQL query should return **only one row**, giving the genre name and the number of songs in the database in that genre.

In [45]:
cur.execute('SELECT genres.Name, COUNT(tracks.TrackId) AS Number_of_Tracks FROM genres JOIN tracks ON tracks.GenreId=genres.GenreId GROUP BY genres.Name ORDER BY Number_of_Tracks ASC LIMIT 1')
for a in cur:
    print(dict(a))

{'Name': 'Opera', 'Number_of_Tracks': 1}


**2.10** What is the *most* popular genre by **number of purchases**? Your SQL query should return **only one row**, giving the genre name and the number of songs in the database in that genre.

In [47]:
cur.execute('SELECT genres.Name, COUNT(invoice_items.TrackId) AS Number_of_Purchases FROM genres JOIN tracks ON tracks.GenreId = genres.GenreId JOIN invoice_items ON invoice_items.TrackId = tracks.TrackId GROUP BY genres.Name ORDER BY Number_of_Purchases DESC LIMIT 1')
for a in cur:
    print(dict(a))






{'Name': 'Rock', 'Number_of_Purchases': 835}


**2.10** How many "Alternative & Punk" songs were purchased in the United States?

In [64]:
cur.execute('SELECT COUNT(*) AS Alt_Punk_Song_Purchase_Count FROM genres JOIN tracks ON tracks.GenreId = genres.GenreId JOIN invoice_items ON invoice_items.TrackId = tracks.TrackId JOIN invoices ON invoices.InvoiceId = invoice_items.InvoiceId WHERE genres.Name = "Alternative & Punk" AND invoices.BillingCountry = "USA"')
for a in cur:
    print(dict(a))

{'Alt_Punk_Song_Purchase_Count': 50}


**2.11** Print out a track listing of the "TV Shows" playlist: Artist name, track name, album name, and Composer

In [68]:
cur.execute('SELECT artists.Name, tracks.Name AS Track, albums.Title AS Album, tracks.Composer FROM artists JOIN albums ON albums.ArtistId = artists.ArtistId JOIN tracks ON tracks.AlbumId = albums.AlbumId JOIN playlist_track ON playlist_track.TrackId = tracks.TrackId JOIN playlists ON playlists.PlaylistId = playlist_track.PlaylistId WHERE playlists.Name = "TV Shows"')
for a in cur:
    print(dict(a))

{'Name': 'Battlestar Galactica', 'Track': 'Battlestar Galactica: The Story So Far', 'Album': 'Battlestar Galactica: The Story So Far', 'Composer': None}
{'Name': 'Battlestar Galactica', 'Track': 'Occupation / Precipice', 'Album': 'Battlestar Galactica, Season 3', 'Composer': None}
{'Name': 'Battlestar Galactica', 'Track': 'Exodus, Pt. 1', 'Album': 'Battlestar Galactica, Season 3', 'Composer': None}
{'Name': 'Battlestar Galactica', 'Track': 'Exodus, Pt. 2', 'Album': 'Battlestar Galactica, Season 3', 'Composer': None}
{'Name': 'Battlestar Galactica', 'Track': 'Collaborators', 'Album': 'Battlestar Galactica, Season 3', 'Composer': None}
{'Name': 'Battlestar Galactica', 'Track': 'Torn', 'Album': 'Battlestar Galactica, Season 3', 'Composer': None}
{'Name': 'Battlestar Galactica', 'Track': 'A Measure of Salvation', 'Album': 'Battlestar Galactica, Season 3', 'Composer': None}
{'Name': 'Battlestar Galactica', 'Track': 'Hero', 'Album': 'Battlestar Galactica, Season 3', 'Composer': None}
{'Name'

## BONUS

**BONUS.1** What is the least popular genre by number of purchases? Your SQL query should return **only one row**, giving the genre name and the number of purchases of songs in that genre.

In [ ]:
# CHANGE ME: insert code here

**BONUS.2** For each country, list how many songs were purchased in each genre in that country. In the same query, also get the percentage of purchases in that country that were made up of each genre. For example, you should have a row for `"Alternative & Punk"` in `"Argentina"`, giving the count of songs (say, `10`) and the percent of all purchases in Argentina made up of Alternative & Punk songs (say, `.30`, i.e., 30%). (Note, these numbers are examples.)

In [ ]:
# CHANGE ME: insert code here

## Close the database

In [81]:
con.close()